In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import requests

from kiwipiepy import Kiwi
from itertools import chain
from collections import Counter
import ast

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#* GPU 사용 시
device = torch.device("cuda:0")

import json
from math import ceil

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU가 사용 중입니다.")
else:
    device = torch.device("cpu")

GPU가 사용 중입니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install gluonnlp pandas tqdm
# !pip install mxnet
# !pip install sentencepiece==0.1.91
# !pip install transformers==4.8.2
# !pip install torch
# !pip install kiwipiepy
# !pip install transformers==2.5.1
# !pip install sentencepiece
# !pip install transformers
# !pip install kobert-transformers

In [ ]:
# !git clone https://github.com/google/sentencepiece.git
# !cd sentencepiece
# !mkdir build
# !cd build
# !cmake ..
# !make -j $(nproc)
# !sudo make install
# !sudo ldconfig -v

In [ ]:
#TODO model과 tokenizer가 서로 mapping이 되어야 함
from transformers import pipeline

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/에어비엔비/감성사전/review_token.csv')
text = data['text'].tolist()

In [ ]:
#TODO model과 tokenizer가 서로 mapping이 되어야 함
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
pipe = pipeline("text-classification", model="nlp04/korean_sentiment_analysis_kcelectra")
tokenizer = AutoTokenizer.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")
model = AutoModelForSequenceClassification.from_pretrained("nlp04/korean_sentiment_analysis_kcelectra")

In [ ]:
pipe = pipeline('text-classification', 'nlp04/korean_sentiment_analysis_kcelectra', return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
outputs = [pipe(text[idx]) for idx in tqdm(range(len(text[:30000])))]
len(outputs)


  0%|          | 101/30000 [00:05<20:12, 24.66it/s]

In [ ]:
output0 = pipe(text[0])

['더할 나위 없이 아파트 자체가 훌륭하고 필요한 모든 것이 갖춰져 있습니다',
 '게다가 동네 자체가 매우 친절하다는 점 외에도 가족 생활의 에너지가 가득합니다 슈퍼마켓과 일부 편의점 및 기차역이 정말 가깝습니다',
 '처음부터 집처럼 느껴질 것입니다 쉽게 구할 수 있는 물건을 포장하다',
 '아파트는 사진에서 보이는 것보다 더 넓습니다',
 '커플에게 적합합니다 깨끗하고 잘 관리되어 있고 안전하며 유용한 정보가 많이 있습니다',
 '서양인으로서 우리는 다다미 바닥에서 이상한 냄새가 난다는 것을 알았지만 나중에 모든 다다미 방에서 같은 냄새가 난다는 것을 알게 되었습니다 지하철역에서 도보로 약 분 거리에 위치 이 노선에서는 패스가 유효하지 않으며 슈퍼마켓 상점과 레스토랑에서 도보 거리에 있습니다',
 '전반적으로 좋은 경험을 했으며 도쿄를 여행하는 모든 사람에게 추천합니다',
 '아파트는 커플에게 적합합니다 조금 작지만 정말 완벽한 가구가 그것을 보완합니다 일상 생활에 필요한 모든 것이 제공됩니다',
 '우리는 또한 동네를 많이 즐겼으며 주요 도시와 가까운 매력적인 장소입니다 머무르게 해주셔서 감사합니다',
 '다시 방문하겠습니다',
 '요시미와 마렉의 사랑스러운 아파트에서 즐거운 시간을 보냈습니다',
 '모든 것이 새로 정리되고 정돈되어 있었으며 우리에게 필요한 모든 편안함과 편리함이 갖춰져 있었습니다',
 '그들이 우리에게 남겨준 생존 키트에는 집과 동네에 대한 정보가 가득했고 긴급 전화번호까지 포함되어 있었습니다',
 '다른 임대 숙소에서는 볼 수 없었던 기타 세부 사항 이 아파트는 미국 표준에 비해 규모가 작은 일본식이지만 모든 것이 잘 설계되어 우리 인 가족이 꽤 편안했습니다',
 '동네가 특별히 그림처럼 아름답지는 않았지만 확실히 편리하고 흥미로웠습니다 훌륭한 가치와 훌륭한 숙박',
 '우리는 오시아게 홀리데이 아파트에서의 숙박을 좋아했습니다 아파트는 매우 좋았고 매우 편안하고 흠집이 없었습니다',
 '스미다강과 아사쿠사가 가깝고 새로운 도쿄 스카

In [ ]:
outputs

[[{'label': '기쁨(행복한)', 'score': 0.6286994218826294},
  {'label': '고마운', 'score': 0.17953263223171234},
  {'label': '설레는(기대하는)', 'score': 0.0020338643807917833},
  {'label': '사랑하는', 'score': 0.0021815800573676825},
  {'label': '즐거운(신나는)', 'score': 0.1524095982313156},
  {'label': '일상적인', 'score': 0.00972855743020773},
  {'label': '생각이 많은', 'score': 0.018288590013980865},
  {'label': '슬픔(우울한)', 'score': 0.0030943171586841345},
  {'label': '힘듦(지침)', 'score': 0.0013047578977420926},
  {'label': '짜증남', 'score': 0.00168660341296345},
  {'label': '걱정스러운(불안한)', 'score': 0.0010400965111330152}]]